In [2]:
import mpmath


mpmath.mp.dps = 50 # setup the solver precision

# characteristics of step-index fibers
n_core = 1.55  # refractive index in core
n_clad = 1.46  # refractive index in cladding
lambda_0 = 1000e-9  # wavelength in meters
R_core = 5e-6  # core radius in meters

# find the modes in the form of TE_0x, TM_0X, HE_mx, EH_mx
mode_type = 'HE'  # TE TM HE EH
m = 1  # for TE and TM mode here is 0

# setup search step, refractive index from cladding to core
search_step = mpmath.mpf('0.01')


class FiberModeSolver:
    def __init__(self, n_core, n_clad, lambda_0, R_core, m):
        self.n_core = mpmath.mpf(n_core)
        self.n_clad = mpmath.mpf(n_clad)
        self.lambda_0 = mpmath.mpf(lambda_0)
        self.R_core = mpmath.mpf(R_core)
        self.k_0 = mpmath.pi * 2 / self.lambda_0
        self.m = mpmath.mpf(m)

    def TE_mode(self, n_eff):
        U = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - n_eff**2)
        W = self.k_0 * self.R_core * mpmath.sqrt(n_eff**2 - self.n_clad**2)
        eq = mpmath.besselj(1, U) / (U * mpmath.besselj(0, U)) + mpmath.besselk(1, W) / (W * mpmath.besselk(0, W))

        return [eq]

    def TM_mode(self, n_eff):
        U = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - n_eff**2)
        W = self.k_0 * self.R_core * mpmath.sqrt(n_eff**2 - self.n_clad**2)
        eq = (n_core**2*mpmath.besselj(1, U)) / (U * mpmath.besselj(0, U)) + (n_clad**2*mpmath.besselk(1, W)) / (W * mpmath.besselk(0, W))
        return [eq]

    
    def HE_mode(self, n_eff):
        n_ratio =  self.n_clad**2 / self.n_core**2
        U = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - n_eff**2)
        W = self.k_0 * self.R_core * mpmath.sqrt(n_eff**2 - self.n_clad**2)
        V = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - self.n_clad**2 )
        
        self.m = mpmath.mpf(m)
        J_prime = mpmath.besselj(self.m, U)/ U - mpmath.besselj(self.m+1, U)
        K_prime = mpmath.besselk(self.m, W)/ W - mpmath.besselk(self.m+1, W)
        
        J = J_prime / (U * mpmath.besselj(self.m, U))
        K = K_prime / (W * mpmath.besselk(self.m, W))

        # "+" the discriminant stand for HE mode
        eq =  2*J + (n_ratio+1)*K + mpmath.sqrt( (1 - n_ratio)**2*K**2 + (4 * n_eff**2 * V**4)/ (n_core**2 * U**4 * W**4))
       
        return [eq]

    def EH_mode(self, n_eff):
        n_ratio =  self.n_clad**2 / self.n_core**2
        U = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - n_eff**2)
        W = self.k_0 * self.R_core * mpmath.sqrt(n_eff**2 - self.n_clad**2)
        V = self.k_0 * self.R_core * mpmath.sqrt(self.n_core**2 - self.n_clad**2 )
        
        self.m = mpmath.mpf(m)
        J_prime = mpmath.besselj(self.m, U)/ U - mpmath.besselj(self.m+1, U)
        K_prime = mpmath.besselk(self.m, W)/ W - mpmath.besselk(self.m+1, W)
        
        J = J_prime / (U * mpmath.besselj(self.m, U))
        K = K_prime / (W * mpmath.besselk(self.m, W))

        # "-" the discriminant stand for EH mode
        eq =  2*J + (n_ratio+1)*K - mpmath.sqrt( (1 - n_ratio)**2*K**2 + (4 * n_eff**2 * V**4)/ (n_core**2 * U**4 * W**4))
        
        return [eq]
    
    def solve(self, mode_type, search_precision):
        results = []
        for n_eff_guess in mpmath.arange(self.n_core - mpmath.mpf('0.001'), self.n_clad - mpmath.mpf('0.001'), - search_step):
        #for n_eff_guess in mpmath.arange(self.n_core - mpmath.mpf('0.001'), 1.52, -0.005):
            #print(f"n_eff guess value: {n_eff_guess}")
            if mode_type == 'TE':
                try:
                    n_eff_solution = mpmath.findroot(self.TE_mode, n_eff_guess)
                    print(f" Solution found : {n_eff_solution}")
                except:
                    pass
            elif  mode_type == 'TM':
                try:
                    n_eff_solution = mpmath.findroot(self.TM_mode, n_eff_guess)
                    print(f" Solution found : {n_eff_solution}")
                except:
                    pass
            elif  mode_type == 'HE':
                try:
                    n_eff_solution = mpmath.findroot(self.HE_mode, n_eff_guess)
                    print(f" Solution found : {n_eff_solution}")
                except:
                    pass
            elif  mode_type == 'EH':
                try:
                    n_eff_solution = mpmath.findroot(self.EH_mode, n_eff_guess)
                    print(f" Solution found : {n_eff_solution}")
                except:
                    pass
            else:
                print(f"unknown_type: {mode_type}")
                break 
            
            #if n_eff_solution not in results:
            #   results.append(n_eff_solution)
            # review repeated value
            try:
                is_unique = True
                for existing_solution in results:
                    existing_value = existing_solution[0,0]
                    n_eff_value = n_eff_solution[0,0]
                    if mpmath.almosteq(n_eff_value, existing_value, rel_eps=1e-15):
                        is_unique = False
                        break
                if is_unique:
                    results.append(n_eff_solution)
            except:
                    pass
        return results





solver = FiberModeSolver(n_core, n_clad, lambda_0, R_core, m)
results = solver.solve(mode_type, search_step)

# print the results
for index, matrix in enumerate(results, start=1):
    value = matrix[0, 0]
    if mode_type == 'HE' or mode_type =='EH':
        print(f" {mode_type}_{m}{index}: {value}")
    elif mode_type == 'TE' or mode_type =='TM':
        print(f" {mode_type}_{0}{index}: {value}")



 Solution found : [1.5483098555191531107593891173489062573573446611759]
 Solution found : [1.5410888789671602178533474183705223199660899655684]
 Solution found : [1.5280842860325287717037349442579112818585864646488]
 Solution found : [1.5280842860325287717037349442579112818585864646488]
 Solution found : [1.5093260686609256537368686471367570931922114036331]
 Solution found : [1.5093260686609256537368686471367570931922114036331]
 Solution found : [1.4850997464589695610459987673645657911370037244755]
 Solution found : [1.4850997464589695610459987673645657911370037244755]
 Solution found : [1.4850997464589695610459987673645657911370037244755]
 HE_11: 1.5483098555191531107593891173489062573573446611759
 HE_12: 1.5410888789671602178533474183705223199660899655684
 HE_13: 1.5280842860325287717037349442579112818585864646488
 HE_14: 1.5093260686609256537368686471367570931922114036331
 HE_15: 1.4850997464589695610459987673645657911370037244755
